Trong tệp này, em thực hiện 2 công việc chính:
- Xử lý các file âm thanh mà nhóm đã thu âm ở bài tập 1
- Trích xuất các đặc trưng MFCC của từng từ (39 đặc trưng gồm MFCC, delta và deltadelta)

# Khai báo thư viện, các đường link cần thiết

In [1]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import fnmatch
import pandas as pd
!pip install pydub
from pydub import AudioSegment

In [3]:
from google.colab import drive
drive.mount("/content/drive")
DATA_ROOT_DIR="/content/drive/MyDrive/Speech/Data_14/"
!ls $DATA_ROOT_DIR
DATA_Ex1="/content/drive/MyDrive/Speech/Ex1-2022/"
!ls $DATA_Ex1
data_dirs=[DATA_Ex1+'14/19020061_DuongNganHa/',
           DATA_Ex1+'14/19021240_NguyenTrongDat/',
           DATA_Ex1+'14/19020066_HoangHuuTung/',
           DATA_Ex1+'14/19021229_DangTrungCuong/'
           ]

Mounted at /content/drive
 01   05   10  '15 (Nhóm có STT 14 trong danh sách)'   7
 02   06   11   16				       89.txt
 03   08   13   1.txt				       99
 04   09   14   28.txt				       Groups.gsheet


In [4]:
!mkdir "/content/drive/MyDrive/Speech/Data_14/xuong"
!mkdir "/content/drive/MyDrive/Speech/Data_14/len"
!mkdir "/content/drive/MyDrive/Speech/Data_14/phai"
!mkdir "/content/drive/MyDrive/Speech/Data_14/trai"
!mkdir "/content/drive/MyDrive/Speech/Data_14/nhay"
!mkdir "/content/drive/MyDrive/Speech/Data_14/ban"
!mkdir "/content/drive/MyDrive/Speech/Data_14/A"
!mkdir "/content/drive/MyDrive/Speech/Data_14/B"

# Xử lý các file âm thanh
Với mục đích trích xuất được các đặc trưng MFCC cho từng từ, em đã thực hiện chia nhỏ các file âm thanh đã thu lần trước thành các file nhỏ mà mỗi file chỉ chứa một đoạn âm có 1 label duy nhất. Việc này được thực hiện lần lượt như sau:
- Định nghĩa hàm cắt âm thanh từ đoạn âm thanh dài, biết điểm start và end
- Cắt các file âm thanh với các nhãn mong muốn (ở bài tập này, em bỏ qua label "sil") và lưu vào các thư mục tương ứng
------
*Cách này không được tối ưu do tốn bộ nhớ để lưu lại các file âm thanh sau khi cắt. Trước đó em đã thử thực hiện trích xuất đặc trưng MFCC trên từng mẩu của cả đoạn âm thanh dài và lưu vào mảng tuy nhiên em gặp lỗi chưa sửa được: nếu không bỏ sil thì tồn tại những đoạn sil quá ngắn không tính được delta và delta2; nếu bỏ sil thì em cần apply việc trích xuất cho từng hàng rồi thêm chúng vào dataframe, em bị báo lỗi kích thước các phần thêm vào không đồng đều*

In [5]:
def split_wav(start,end,dir_audio,dir_export,name_export):
    start = start * 1000 
    end = end * 1000
    newAudio = AudioSegment.from_wav(dir_audio)
    newAudio = newAudio[start:end]
    name_export = dir_export + name_export +".wav"
    newAudio.export(name_export, format="wav")

In [6]:
class_names = {'len':0, 'xuong':0, 'trai':0, 'phai':0, 'nhay':0,'ban':0,'A':0,'B':0}

#Với từng thư mục thu âm của mỗi sinh viên
for data_dir in data_dirs:
  files = fnmatch.filter(os.listdir(data_dir), '*.txt')
  files_name = [f.split(".")[0] for f in files]

  #Với từng file âm thanh
  for path in files_name:
    path_wav = path + ".wav"
    path_txt = path + ".txt"
    df_i = pd.read_csv(data_dir+path_txt, sep="\t", header=None)

    #Với từng label
    for index,row in df_i.iterrows():
      if row[2] in class_names:
        class_names[row[2]] += 1
      else:
        continue
        
      #Thực hiện cắt file
      name_export = row[2]+"_"+ str(class_names[row[2]])
      split_wav(row[0],row[1],data_dir+path_wav,""+DATA_ROOT_DIR+row[2]+"/",name_export)

In [7]:
class_names

{'A': 487,
 'B': 539,
 'ban': 473,
 'len': 501,
 'nhay': 545,
 'phai': 456,
 'trai': 505,
 'xuong': 474}

# Trích xuất đặc trưng MFCC
Em thực hiện trích xuất đặc trưng MFCC với lần lượt các bước:
- Định nghĩa hàm trích xuất đặc trưng MFCC, delta, delta2 từ file âm thanh đầu vào. Trong hàm này em thực hiện việc chuẩn hoá luôn (em lựa chọn đưa về [0,1])
- Trích xuất đặc trưng cho từng file cắt được bên trên, nối chúng vào list chung
- Lưu từng list MFCC đã có vào tệp để các mô hình sau gọi lại sử dụng mà không cần tính toán

In [8]:
def extract_mfcc_features(file_path,n_mfcc):
    #load file âm thanh
    sound, sr = librosa.load(file_path)

    #trích xuất đặc trưng
    mfcc = librosa.feature.mfcc(y=sound, sr=sr, n_mfcc=n_mfcc,hop_length=512)
    delta_mfcc = librosa.feature.delta(mfcc,width=9)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)

    #chuẩn hoá
    mfcc_features = np.concatenate((mfcc, delta_mfcc, delta2_mfcc)).T
    min_features = np.min(mfcc_features, axis=0)
    max_features = np.max(mfcc_features, axis=0)
    mfcc_features_nom = (mfcc_features - min_features) / (max_features - min_features)
    
    return mfcc_features_nom

In [10]:
class_names_arr = ["len", "xuong", "phai", "B", "A", "trai", "nhay", "ban"]
mfcc_features_data = {}
sr=22050
n_mfcc=13

#Với từng label
for cname in class_names_arr:
    mfcc_features_data[cname] = []

    #Xét tất cả các mẫu đã có
    file_paths = [os.path.join("",DATA_ROOT_DIR, cname, file_name) for file_name in fnmatch.filter(os.listdir(os.path.join("",DATA_ROOT_DIR, cname)), '*.wav')]
    for file_path in file_paths:
      mfcc = extract_mfcc_features(file_path,n_mfcc)
      
      #Bỏ qua nan để tránh lỗi khi sử dụng cho model
      if np.isnan(np.sum(mfcc)):
        print(file_path)
        continue
        
      #Thêm vào list chung  
      mfcc_features_data[cname].append(mfcc)

In [14]:
mfcc_features_data['A'][0].shape

(23, 39)

In [15]:
#Lưu các giá trị tính được ra file
for cname in class_names:
  np.save(DATA_ROOT_DIR+"MFCC/"+cname+"_mfcc",mfcc_features_data[cname])

/usr/local/lib/python3.7/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
